# Dependence of trajectories on $j$ 

We approximate the survival function of the Gamma distribution with a hypoexponential distribution, and a system of ODEs. How does the solution of ODEs depend on $j$?

In [ ]:
import numpy as np
import scipy.stats as sts
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.transforms import blended_transform_factory

In [ ]:
plt.rcParams.update({'font.size': 18})

In [ ]:
def ode_sys_fixed(t, u, n, j, tau):
    gamma = n/tau
    frj = j - np.floor(j)
    x = n/tau / (1 + np.sqrt(n/(2*j) * (1-frj)))
    y = n/tau / (1 - np.sqrt(n/(2*j) * (1-frj)))
    du = np.zeros(n)
    if n > 2:
        du[0] = -gamma * u[0]
        for k in range(1,n-2):
            du[k] = gamma * (u[k-1] - u[k])
        du[n-2] = gamma * u[n-3] - x * u[n-2]
        du[n-1] = x * u[n-2] - y * u[n-1]
    elif n == 2:
        du[0] = -x * u[0]
        du[1] = x * u[0] - y * u[1]
    else:
        raise Exception("ERROR!")
    return du

def ode_sys_smooth(t, u, n, j, tau, h):
    gamma = j/tau
    frj = j - np.floor(j)
    x = 1/(tau/(2*j) * (1 + frj + np.sqrt(1 - frj**2 + h**2) - h))
    y = 1/(tau/(2*j) * (1 + frj - np.sqrt(1 - frj**2 + h**2) + h))
    du = np.zeros(n)
    if n > 2:
        du[0] = -gamma * u[0]
        for k in range(1,n-2):
            du[k] = gamma * (u[k-1] - u[k])
        du[n-2] = gamma * u[n-3] - x * u[n-2]
        du[n-1] = x * u[n-2] - y * u[n-1]
    elif n == 2:
        du[0] = -x * u[0]
        du[1] = x * u[0] - y * u[1]
    else:
        raise Exception("ERROR!")
    return du

## exact solution
def u_exact(t, j, tau):
    return sts.gamma.sf(t, j, scale=tau/j)

In [ ]:
js = np.linspace(1.01, 6.1, 1690)

tmax = 4
tau = 3.0
h = 1e-2

vals_n = np.zeros(len(js))
vals_j = np.zeros(len(js))

for i, j in enumerate(js):
    n = max(2, int(np.ceil(j)))
    u0 = np.zeros(n)
    u0[0] = 1
    t_span = (0, tmax)
    u_n = solve_ivp(lambda t, u: ode_sys_fixed(t, u, n, j, tau), t_span, u0)
    u_j = solve_ivp(lambda t, u: ode_sys_smooth(t, u, n, j, tau, h), t_span, u0)
    vals_n[i] = np.sum(u_n.y[:,-1])
    vals_j[i] = np.sum(u_j.y[:,-1])

## Figure for manuscript

In [ ]:
def hybrid_trans(ax):
    return blended_transform_factory(ax.transData, ax.transAxes)


fig = plt.figure(figsize=(10,5))

gs = GridSpec(1,2)

ax = fig.add_subplot(gs[:,1])

ax.plot(js, vals_n, linewidth=1, color='k', zorder=2, label="fixed")
ax.plot(js, vals_j, linewidth=4, color='lightcoral', zorder=1, label="smoothed")
ax.plot(js, [u_exact(tmax, j, tau) for j in js], color='tab:blue', label="exact")


ax.legend(fontsize='x-small')

ax.set_xlabel("$j$")
ax.set_ylabel("$u^j(t_1)$")

j = 2.1

ax.axvline(x=j, color='k', linestyle='--')
ax.text(j, 1, "$j*$", transform=hybrid_trans(ax), ha='center', va='bottom')

n = max(2, int(np.ceil(j)))
u0 = np.zeros(n)
u0[0] = 1
t_span = (0, tmax+1)
t_eval = np.linspace(*t_span, 1000)
u_n = solve_ivp(lambda t, u: ode_sys_fixed(t, u, n, j, tau), t_span, u0, t_eval=t_eval)
u_j = solve_ivp(lambda t, u: ode_sys_smooth(t, u, n, j, tau, h), t_span, u0, t_eval=t_eval)

bx = fig.add_subplot(gs[:,0])

bx.plot(t_eval, np.sum(u_n.y, axis=0), linewidth=1, color='k', zorder=2)
bx.plot(t_eval, np.sum(u_j.y, axis=0), linewidth=4, color='lightcoral', zorder=1)

#bx.plot(t_eval, [u_exact(t, j, tau) for t in t_eval], color='tab:blue', zorder=0, linewidth=6)

bx.set_xlabel("$t$")
bx.set_ylabel("$u^{j*}(t)$")

bx.axvline(x=tmax, color='k', linestyle='--')
bx.text(tmax, 1, "$t_1$", transform=hybrid_trans(bx), ha='center', va='bottom')
bx.set_ylim(-0.05,1.05)

bx.text(-0.1, 1.1, 'A', fontsize='x-large', transform=bx.transAxes)
ax.text(-0.1, 1.1, 'B', fontsize='x-large', transform=ax.transAxes)


fig.tight_layout()

fig.savefig("../sensitivity.pdf", bbox_inches='tight')